# ATTACHMENT 1: Financial Analysis of Vail Resorts Inc. (MTN)

## Preliminary Code
First of all, we import relevant libraries and extract the company's financial data useful for the subsequent analysis.

In [1]:
# Loading useful libraries
import yfinance as yf
import pandas as pd
import numpy as np

# Extracting historical stock prices thorugh Yahoo Finance
ticker = 'MTN' # Vail Resorts Inc.
stock_data = yf.download(ticker, start='1997-03-01', end='2024-04-01')
stock_5years = yf.download(ticker, start='2019-04-01', end='2024-04-01')

# Extracting the company's general financial data
company = yf.Ticker(ticker)
balance_sheet = company.balance_sheet
financials = company.financials
#tax_rate = financials['2023-12-31'].loc['Tax Rate For Calcs'] (We used this initially and then changed it to the US Corporate tax rate)
tax_rate = .21
cashflow = company.cashflow
dividends = company.dividends
market_cap = company.info['marketCap']
enterprise_value = company.info['enterpriseValue']
revenue = financials.loc['Total Revenue'].iloc[0]
operating_income = financials.loc['Operating Income'].iloc[0]
capex = cashflow.loc['Capital Expenditure Reported'].iloc[0]
nwc = balance_sheet.loc['Current Assets'] - balance_sheet.loc['Current Liabilities']

# Visualizing stock data format
print("\nStock data (last 5 rows):\n", stock_data.tail())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Stock data (last 5 rows):
                   Open        High         Low       Close   Adj Close  Volume
Date                                                                          
2024-03-22  225.139999  225.139999  220.940002  221.500000  219.251785  277400
2024-03-25  221.309998  221.309998  217.759995  218.770004  216.549500  432300
2024-03-26  221.729996  221.729996  217.820007  218.720001  216.500000  426400
2024-03-27  217.740005  220.270004  217.080002  220.179993  220.179993  349900
2024-03-28  220.199997  224.119995  219.229996  222.830002  222.830002  347700


## Cost of Equity Estimation
We compute the Cost of Equity with two different methods: as the sum of Dividend Growth Rate and Dividend Yield and using the CAPM formula.

In [9]:
# Estimating the cost of equity based on the dividend growth and yield
div_growth = dividends.pct_change().mean()
div_yield = dividends.mean() / stock_data['Close'].mean()
cost_of_equity = div_growth + div_yield
print(f"\n\nEstimated Dividend Growth Rate = {div_growth:.2%}")
print(f"Dividends-estimated Cost of Equity = {cost_of_equity:.2%}")

# Extracting S&P 500 index's data
historical_data_SaP = yf.download('SPY', start='2014-03-01', end='2024-04-01')

# Estimating our own beta to be used in the CAPM
market_prices2 = yf.download('SPY', start='2019-04-01', end='2024-04-01')['Adj Close']

# Extracting the risk-free rate based on the 10-year Treasury Note
treasury = yf.Ticker("^TNX")
risk_free_rate = treasury.history(period="10y")['Close'].iloc[-1] / 100 # Converting basis points to decimal
print(f"\nEstimated Risk-Free Rate = {risk_free_rate:.2%}")

# Getting historical returns for the S&P 500
historical_data = historical_data_SaP['Adj Close']
historical_data = historical_data.dropna()
market_return = np.log(historical_data/historical_data.shift(1))
market_return.dropna(inplace = True)
annual_return = market_return.mean()*252
print(f"Estimated Annual Market Return = {annual_return:.2%}")

# Extracting the daily returns
stock_returns2 = stock_5years['Adj Close'].pct_change().dropna()
market_returns2 = market_prices2.pct_change().dropna()

# Calculating beta as stock-market return covariance divided by market returns' variance
covariance = np.cov(stock_returns2, market_returns2)[0][1]
market_variance = np.var(market_returns2)
beta2 = covariance / market_variance

# Estimating the cost of equity using the CAPM formula
cost_of_equity2 = risk_free_rate + beta2 * (annual_return - risk_free_rate)
print(f"CAPM-estimated Cost of Equity = {cost_of_equity2:.2%}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Estimated Dividend Growth Rate = 8.43%
Dividends-estimated Cost of Equity = 9.49%

Estimated Risk-Free Rate = 4.71%
Estimated Annual Market Return = 12.16%
CAPM-estimated Cost of Equity = 11.70%


We obviously obtain two different values; the dividend-estimated Cost of Equity in particular will be used for the DCF Perpetuity analysis of the Stock Price.

## P/E and P/B Calculation

We estimate the Price-to-Earnings and Price-to-Book Ratios considering data from the company's IPO, then we compare the results with the values extracted via Yahoo Finance and with the performance indicators of some MTN's market competitors; this analysis might be useful in determining the Stock Price's fairness.

In [3]:
# Computing the price-to-earnings ratio (per-share values considered)
net_income = financials.loc['Net Income'].iloc[0]
earning_per_share = net_income / balance_sheet.loc['Share Issued'].iloc[0]
price_per_share = stock_data['Close']
P_E_ratio = price_per_share / earning_per_share
print(f"Estimated P/E Ratio = {P_E_ratio.iloc[-1]:.3f}")

# Extracting the same ratio from Yahoo Finance for comparison
print(f"Yahoo-Finance P/E Ratio = {company.info['trailingPE']:.3f}")


# Computing the market-price to book-value ratio
book_value_ps = balance_sheet.loc['Total Equity Gross Minority Interest'].iloc[0] / balance_sheet.loc['Share Issued'].iloc[0]
P_B_ratio = price_per_share / book_value_ps
print(f"\nEstimated P/B Ratio = {P_B_ratio.iloc[-1]:.3f}")

# Extracting the same ratio from Yahoo Finance for comparison
print(f"Yahoo-Finance P/B Ratio = {company.info['priceToBook']:.3f}")

Estimated P/E Ratio = 38.889
Yahoo-Finance P/E Ratio = 32.515

Estimated P/B Ratio = 7.807
Yahoo-Finance P/B Ratio = 9.103


### Comparative Analysis of Ratios

MSN Money already did an analysis on Vail Resorts' Price-over-Earnings and Price-to-Book, reporting end-January results (P/E: 37.06, P/B: 10.18) not much different from ours. The small discrepancies must be due to the long-period data aggregation we operated; similarly, the difference between our estimations and Yahoo Finance's ratios is explainable by the fact that the latter are 12-month trailing values.

Moreover, compared to the competitors identified by MSN Money (i.e., CCL-Carnival Corp, LVS-Las Vegas Sand Corp and NCLH-Norwegian Cruise Line Holding Ltd), we notice that MTN presented quite positive indicators for F.Y. 2023: the focus firm had the second highest P/E score - of 34.92, compared to NCLH's record one of 51.54 - and a high P/B too - of 9.33, compared to NCLH's 28.47. It is also important to notice that the sector in which Vail Resorts operates is rather a niche one, thus it is impossible to draw an actually direct comparison with other companies.

(Source: https://www.msn.com/en-us/money/stockdetails/fi-a1y3dm?id=a1y3dm&l3=L3_Analysis)

## Stock Price Estimation
We estimate the Stock Price by two methods:

1) Projected Free Cash Flows system;

2) Discounted Cash Flow Perpetuity model.

In [10]:
# 1) Projected Free Cash Flows

# Calculating financial metrics
revenue = financials.loc['Total Revenue'].iloc[0]
operating_income = financials.loc['Operating Income'].iloc[0]
capex = cashflow.loc['Capital Expenditure'].iloc[0]
nwc = balance_sheet.loc['Current Assets'] - balance_sheet.loc['Current Liabilities']
operating_margin = (operating_income / revenue) * 100
capex_percentage_of_revenue = (capex / revenue) * 100
change_in_nwc_percentage_of_revenue = ((nwc.iloc[0] - nwc.iloc[-1]) / revenue) * 100
print("Operating Margin:", round(operating_margin, 2), "%")
print("Capex Percentage of Revenue:", round(capex_percentage_of_revenue, 2), "%")
print("Change in NWC Percentage of Revenue:", round(change_in_nwc_percentage_of_revenue, 2), "%")

# Calculating the revenue growth and prediciting the future revenue (with gradual decrease of g to 6%)
revenue_growth = financials.loc['Total Revenue'].iloc[::-1].pct_change()
revenue_growth.dropna(inplace=True)
last_known_growth = revenue_growth.iloc[-1]
future_years = 6  # Number of future years to predict
final_growth_rate = 0.06  # Target final growth rate

# Creating a linear interpolation between the last known growth and the final target growth
predicted_growth = np.linspace(last_known_growth, final_growth_rate, future_years)

# Projecting future revenues
current_revenue = financials.loc['Total Revenue'].iloc[0]  # Revenue current year
projected_revenues = [current_revenue * ((1 + predicted_growth) ** year) for year in range(1, 6)]

# Projecting future free cash flows
free_cash_flows = []
for revenue in projected_revenues:
    operating_income = revenue * operating_margin
    capex = revenue * capex_percentage_of_revenue
    change_in_nwc = revenue * change_in_nwc_percentage_of_revenue
    free_cash_flow = operating_income - capex - change_in_nwc
    free_cash_flows.append(free_cash_flow)

# Estimating the stock price
w_avg_cost_of_capital = 0.0989 # Obtained from future calculations for the WACC
debt = balance_sheet.loc['Long Term Debt'].iloc[0]
present_value_flows = sum(cf / ((1 + w_avg_cost_of_capital) ** (i+1)) for i, cf in enumerate(free_cash_flows))
shares_outstanding = balance_sheet.loc['Share Issued'].iloc[0]  # Assuming outstanding shares
stock_price = (present_value_flows - debt) / shares_outstanding
stock_price = stock_price.mean()
print(f"Future-CFs-estimated Stock Price = {stock_price:.2f}$")

Operating Margin: 19.49 %
Capex Percentage of Revenue: -10.9 %
Change in NWC Percentage of Revenue: 0.55 %
Future-CFs-estimated Stock Price = 253.35$


Using the projected future Free Cash Flows system, we build a Stock Price estimation which highly relies on the future Revenue Growth assessment; we estimated this value through iterative tests and in order to potentially achieve a Stock Price coherent with the actual one. It is also important to notice that we assumed the WACC equal to the one we estimated (dedicated code section below).

The obtained Stock Price (253.35\$) is a bit higher than the actual one; this discrepancy is possibly due to a predicted Revenue Growth imprecision.

In [5]:
# 2) Discounted Cash Flow Perpetuity

# Computing the present value of the perpetuity of stock using the DCF
div_future = dividends[46]*(1+div_growth)
pvperp = div_future / (cost_of_equity-div_growth)
print(f"DCF-Perpetuity-estimated Stock Price = {pvperp:.2f}$")

DCF-Perpetuity-estimated Stock Price = 226.08$


However, the Stock Price computed via the DCF perpetuity model - with Cost of Equity = 9.49% and Dividend Growth Rate = 8.43% - returns a value (226.08\$) very similar to the market one as of 14/04/2024 (i.e., 226.97\$), which indicates a rather fair Stock Price.

### Recent Substantial Price Drop

As of the end of April 2024, MTN's Price had a significant decrease, reaching a value of around 200\$. Recent analyses suggest that this drop (-10.6% in 5 days) might have been caused by the slowing Rates of Return and the recent price cut of Morgan Stanley from 242\$ to 229\$. It is inferable that our DCF-estimated Stock Price was slightly more accurate than the one calculated basing on Projected Free Cash Flows.

(Source: https://www.defenseworld.net/2024/04/23/vail-resorts-nysemtn-price-target-cut-to-229-00-by-analysts-at-morgan-stanley.html)

## ROE and Plowback Estimation

In [6]:
# Computing the ROE and plowback
roe = net_income / balance_sheet.loc['Total Equity Gross Minority Interest'].iloc[0]
print(f"Estimated Return On Equity = {roe:.2f}")
retained_earnings = balance_sheet.loc['Retained Earnings'].iloc[0]
total_revenue = financials.loc['Total Revenue'].iloc[0]
plowback = retained_earnings / total_revenue
payout = 1 - plowback
print(f"Estimated Plowback Ratio = {plowback:.2f}")

Estimated Return On Equity = 0.20
Estimated Plowback Ratio = 0.30


## Beta Computation
We compute the Beta manually and then extract it via Yahoo Finance; conscious that the latter value represents the monthly 5-year Beta.

In [7]:
# Extracting market's and stock's daily returns
stock_5years = yf.download(ticker, start='2019-04-01', end='2024-04-01')
market_prices = yf.download('SPY', start='2019-04-01', end='2024-04-01')['Adj Close']
stock_returns = stock_5years['Adj Close'].pct_change().dropna()
market_returns = market_prices.pct_change().dropna()

# Calculating beta as stock-market return covariance divided by market returns' variance
covariance = np.cov(stock_returns, market_returns)[0][1]
market_variance = np.var(market_returns)
beta = covariance / market_variance
print(f"\n\nEstimated Beta = {beta:.3f}")

# Extracting the beta from YFinance for comparison
beta_yahoo = company.info['beta']
print("Yahoo-Finance Beta =", beta_yahoo)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed



Estimated Beta = 0.939
Yahoo-Finance Beta = 1.108


The computed Beta (0.939) indicates that the company has a slightly lower Volatility compared to the overall market: if the market goes up or down, the Stock Price is expected to go up or down as well, but to a slightly lesser degree than the market itself. This suggests that the firm has a risk profile slightly below the market average, making it a somewhat less risky investment than the market as a whole.

## Cost of Capital Assessment

We compute the after-tax Weighted Average Cost of Capital: firstly we calculate the Cost of Debt, then the WACC itself using the Cost of Debt and the CAPM-estimated Cost of Equity.

In [8]:
# Calculating the cost of debt
interest_expense = financials.loc['Interest Expense'].iloc[0]
total_debt = balance_sheet.loc['Total Debt'].iloc[0]
cost_of_debt = interest_expense / total_debt
print(f"Estimated Cost of Debt = {cost_of_debt:.2%}")

# Estimating the WACC after tax
total_debt = enterprise_value - market_cap
E_V = market_cap / enterprise_value
D_V = total_debt / enterprise_value
wacc = (E_V * cost_of_equity2) + (D_V * cost_of_debt * (1 - tax_rate))
print(f"Estimated WACC = {wacc:.2%}")

Estimated Cost of Debt = 5.06%
Estimated WACC = 9.73%


In order to assess their fairness, we compare our calculations to those of GuruFocus (updated to the end of April 2024). Firstly, our CAPM-estimated Cost of Equity (11.70%) is very similar to the aforementioned website's one (i.e., 9.42%), also computed via the CAPM formula. An even lower discrepancy is found comparing the Cost of Debt estimations, as Gurufocus' value is of 5.24% and ours of 5.06%. The key difference in this case is the Tax Rate: they calculated it to be 26.0%, while we used the US Corporate Tax Rate (i.e., 21%). This led the website's computations to a WACC of 7.86%, which is not very similar to ours (9.77%), possibly also due to the CAPM-estimated Cost of Equity.

(Source: https://www.gurufocus.com/term/wacc/MTN#:~:text=cost%20of%20debt.-,As%20of%20Jan.,158.965%20%2F%203032.6538%20%3D%205.2418%25)